# ```H3``` Tutorial #

### What is ```H3```? ###
```H3``` is a geospatial indexing system that partitions the world into hexagonal cells. ```H3``` includes functions for converting from latitude/longitude coordinates to hexagonal cells (at different resolutions), finding their centers, boundary geometry, neighbors, etc.

### Instalation ###
```H3``` can be installed via ```pip3 install h3```. We'll use version ```3.x``` (despite version ```4.x``` being available). 

### Terminology ###
- ```H3Index```: are 15/16-character hexadecimal strings and can represent any ```H3``` object (hexagon, pentagon, directed edge, ...);
- ```Cell```: a geometric/geographic unit polygon in the ```H3``` grid (corresponding to an ```H3Index```) with a given ```resolution```;
- ```Directed Edge```: represents an edge that allows encoding (we can define which cell is the origin and which is the destination);
- ```Grid```: graph with nodes corresponding to ```Cells``` and ```Edges``` given by pairs of adjacent ```Cells```;
- ```Lat/Lng Point```: representation of a geographic point;

### Resolution ###
```Resolution``` define the size of ```Cells```. There are always exactly 12 pentagons at every resolution.

| Resolution | Total number of cells | Number of hexagons  | Number of pentagons |
|------------|-----------------------|---------------------|---------------------|
| 0          | 122                   | 110                 | 12                  |
| 1          | 842                   | 830                 | 12                  |
| 2          | 5,882                 | 5,870               | 12                  |
| 3          | 41,162                | 41,150              | 12                  |
| 4          | 288,122               | 288,110             | 12                  |
| 5          | 2,016,842             | 2,016,830           | 12                  |
| 6          | 14,117,882            | 14,117,870          | 12                  |
| 7          | 98,825,162            | 98,825,150          | 12                  |
| 8          | 691,776,122           | 691,776,110         | 12                  |
| 9          | 4,842,432,842         | 4,842,432,830       | 12                  |
| 10         | 33,897,029,882        | 33,897,029,870      | 12                  |
| 11         | 237,279,209,162       | 237,279,209,150     | 12                  |
| 12         | 1,660,954,464,122     | 1,660,954,464,110   | 12                  |
| 13         | 11,626,681,248,842    | 11,626,681,248,830  | 12                  |
| 14         | 81,386,768,741,882    | 81,386,768,741,870  | 12                  |
| 15         | 569,707,381,193,162   | 569,707,381,193,150 | 12                  |

### Documentation ###
- H3 Documentation - [Link](https://h3geo.org/docs/api/indexing);
- H3 Notebooks - [Link](https://nbviewer.org/github/uber/h3-py-notebooks/tree/master/notebooks/);
- H3 Analysis - [Link](https://towardsdatascience.com/uber-h3-for-data-analysis-with-python-1e54acdcc908);

---

### Importing Dependencies ###

In [23]:
import yaml
import json
import pandas as pd

import folium
from h3 import h3

### User-Defined Functions ###

In [31]:
def build_folium_map(): # Creates 'vanilla' default tile map
    fmap = folium.Map(location= (-22.9725, -43.2116), # 'location': center's lat/lng
                    width= "100%", height="100%", # 'width'/'height': width/height of the map
                    tiles= "cartodb positron", # 'tiles': map tileset (if custom URL, 'attr' is required)
                    zoom_start= 14) # 'zoom_start': initial zoom level
    return fmap

### Main ###

In [24]:
# Reading the YAML config
with open("./h3.yaml", "r") as _stream:
    yaml_config = yaml.safe_load(_stream)

**Indexing**: used for finding ```Cell``` index containing coordinates (as well as its center and boundary);

In [18]:
# 'geo_to_h3(lat, lng, resolution)': indexes location ('lat'/'lng') at given 'resolution' and returns Cell
h3.geo_to_h3(-22.971455797974922, -43.21058220306561, 9)

'89a8a078dcbffff'

In [26]:
# 'h3_to_geo(index)': returns the center of 'index'
h3.h3_to_geo("89a8a078dcbffff")

(-22.97137349189261, -43.20978079691002)

In [63]:
# Creating a default tile map
fmap = build_folium_map()
# 'h3_to_geo_boundary(index)': returns boundaries of 'index'
boundary = h3.h3_to_geo_boundary("89a8a078dcbffff", geo_json=False) # Important: 'geo_json' inverts lat/lng
# Assigning to Polygon
folium.Polygon(locations= boundary, # 'location': no need to add first point at last
               fill= True, # 'kwargs'
               color="blue").add_to(fmap) # 'kwargs'
fmap

**Inspection**: provides metadata about an ```H3``` index;

In [34]:
# 'h3_get_resolution(index)': returns the resolution of 'index'
h3.h3_get_resolution("89a8a078dcbffff")

9

In [35]:
# 'h3_get_base_cell(index)': returns the 'index's 'Cell' number
h3.h3_get_base_cell("89a8a078dcbffff")

84

In [36]:
# 'string_to_h3(index)': converts string 'index' representation to 'H3Index'
h3.string_to_h3("89a8a078dcbffff")

619959663765225471

In [37]:
# 'h3_to_string(h3_index)': converts 'H3Index' to string 'index' representation
h3.h3_to_string(619959663765225471)

'89a8a078dcbffff'

In [42]:
# 'h3_is_valid(index)': returns True if 'index' is valid
h3.h3_is_valid("89a8a078dcbffff")

True

In [43]:
# 'h3_is_pentagon(index)': returns True if 'index' is a pentagonal 'Cell'
h3.h3_is_pentagon("89a8a078dcbffff")

False

In [44]:
# 'h3_get_faces(index)': returns array with all icosahedron faces intersected by 'index'
h3.h3_get_faces("89a8a078dcbffff")

{8}

**Traversal**: allows finding ```Cell```s in the vicinity of origin ```Cell```;

In [82]:
# Creating a default tile map
fmap = build_folium_map()
# 'k_ring(origin, k)': produces 'indexes' within 'k' distance of 'origin' index (k=0)
ring_lst = list(h3.k_ring("89a8a078dcbffff", 1)) # Returns includes 'origin'
# Generating boundaries and adding to map
for index in ring_lst:
    _boundary = h3.h3_to_geo_boundary(index, geo_json=False)
    folium.Polygon(locations= _boundary, # 'location': no need to add first point at last
                   fill=True, # 'kwargs'
                   color="blue").add_to(fmap) # 'kwargs'
fmap

In [77]:
# Defining colors
color_dict = ["blue", "green", "orange", "red"]
# Creating a default tile map
fmap = build_folium_map()
# 'k_ring_distances(origin, k)': similar to 'k_ring()' (above) but with indexes returned in levels ('k'-based)
ring_lst = h3.k_ring_distances("89a8a078dcbffff", 2) # Returns includes 'origin'
# Generating boundaries and adding to map
for idx, level in enumerate(ring_lst):
    for index in level:
        _boundary = h3.h3_to_geo_boundary(index, geo_json=False)
        folium.Polygon(locations= _boundary, # 'location': no need to add first point at last
                       fill=True, # 'kwargs'
                       color=color_dict[idx]).add_to(fmap) # 'kwargs'
fmap

In [88]:
# Creating a default tile map
fmap = build_folium_map()
# 'hex_ring(origin, k)': produces the hollow hex ring of 'indexes' within 'k' distance of 'origin' index (k=0)
ring_lst = list(h3.hex_ring("89a8a078dcbffff", 2)) # Returns doesnt include 'origin'
# Generating boundaries and adding to map
for index in ring_lst:
    _boundary = h3.h3_to_geo_boundary(index, geo_json=False)
    folium.Polygon(locations= _boundary, # 'location': no need to add first point at last
                   fill=True, # 'kwargs'
                   color="green").add_to(fmap) # 'kwargs'
fmap

In [91]:
# 'h3_distance(index1, index2)': returns distance (in grid 'Cell's) between 'index1' and 'index2'
h3.h3_distance("89a8a078dcbffff","89a8a078c23ffff") 

2

**Hierarchy**: allows moving between resolutions (functions produce ```parent```/```children```);

In [96]:
# Creating a default tile map
fmap = build_folium_map()
# 'h3_to_parent(index, parent_res)': returns parent index (at 'parent_res') containing 'index'
parent_idx = h3.h3_to_parent("89a8a078dcbffff", 8)  
# Generating boundaries
child_boundary = h3.h3_to_geo_boundary("89a8a078dcbffff", geo_json=False)
parent_boundary = h3.h3_to_geo_boundary(parent_idx, geo_json=False)
# Adding to map
folium.Polygon(locations= child_boundary, # 'location': no need to add first point at last
               fill=True, # 'kwargs'
               color="blue").add_to(fmap) # 'kwargs'
folium.Polygon(locations= parent_boundary, # 'location': no need to add first point at last
               fill=True, # 'kwargs'
               color="green").add_to(fmap) # 'kwargs'
fmap

In [97]:
# Creating a default tile map
fmap = build_folium_map()
# 'h3_to_children(index, child_res)': returns children indexes (at 'child_res') containing 'index'
child_lst = h3.h3_to_children("89a8a078dcbffff", 10)  
# Generating boundaries and adding to map (parent)
parent_boundary = h3.h3_to_geo_boundary("89a8a078dcbffff", geo_json=False)
folium.Polygon(locations= parent_boundary, # 'location': no need to add first point at last
               fill=True, # 'kwargs'
               color="green").add_to(fmap) # 'kwargs'
# Generating boundaries and adding to map (children)
for index in child_lst:
    _boundary = h3.h3_to_geo_boundary(index, geo_json=False)
    folium.Polygon(locations= _boundary, # 'location': no need to add first point at last
                   fill=True, # 'kwargs'
                   color="blue").add_to(fmap) # 'kwargs'
fmap

In [99]:
# Creating a default tile map
fmap = build_folium_map()
# 'h3_to_center_child(index, child_res)': returns center child 'index' (at 'child_res') 
center_child = h3.h3_to_center_child("89a8a078dcbffff", 10)  
# Generating boundaries
parent_boundary = h3.h3_to_geo_boundary("89a8a078dcbffff", geo_json=False)
child_boundary = h3.h3_to_geo_boundary(center_child, geo_json=False)
# Adding to map
folium.Polygon(locations= child_boundary, # 'location': no need to add first point at last
               fill=True, # 'kwargs'
               color="blue").add_to(fmap) # 'kwargs'
folium.Polygon(locations= parent_boundary, # 'location': no need to add first point at last
               fill=True, # 'kwargs'
               color="green").add_to(fmap) # 'kwargs'
fmap

**Edges**: allows encoding directed edges from one ```Cell``` to a neighboring one;

In [108]:
# 'h3_indexes_are_neighbors(origin_index, destination_index)': returns wheter 'origin_index' and 'destination_index' are neighobrs
h3.h3_indexes_are_neighbors("89a8a078dcbffff", "89a8a078c27ffff")

True

**Misc**


In [109]:
# 'hex_area(resolution, unit)': average hexagon area in 'unit' at 'resolution'
h3.hex_area(9, unit="km^2") # 'unit': 'km^2', 'm^2'

0.1053325

In [110]:
# 'cell_area(index, unit)': specific 'index' area in 'unit'
h3.cell_area("89a8a078dcbffff", unit="km^2") # 'unit': 'km^2', 'm^2'

0.10843249095896922

In [112]:
# 'edge_length(resolution, unit)': average hexagon edge length in 'unit' at 'resolution'
h3.edge_length(9, unit="km") # 'unit': 'km', 'm'

0.174375668

In [113]:
# 'num_hexagons(resolution)': number of 'index's at 'resolution'
h3.num_hexagons(9)

4842432842

In [114]:
# 'get_pentagon_indexes(resolution)': all pentagons 'index's at 'resolution'
h3.get_pentagon_indexes(9)

{'89080000003ffff',
 '891c0000003ffff',
 '89300000003ffff',
 '894c0000003ffff',
 '89620000003ffff',
 '89740000003ffff',
 '897e0000003ffff',
 '89900000003ffff',
 '89a60000003ffff',
 '89c20000003ffff',
 '89d60000003ffff',
 '89ea0000003ffff'}

In [116]:
# 'point_dist(point1, point2, unit)': Haversine distance between 'point1' and 'point2'
h3.point_dist((-22.962775602407085, -43.204551142437545), (-22.9782644648521, -43.21815530689237), unit="km") # 'unit': 'km', 'm'

2.21496335540049

---